In [1]:
import numpy as np
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from alexnet import AlexNet
import time

In [2]:
# PARAMETERS:
BATCH_SIZE = 128
EPOCHS = 10

In [3]:
# Load data
with open('train.p', mode='rb') as f:
    train = pickle.load(f)
X_data, y_data = train['features'], train['labels']

In [4]:
# Split train and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.20, random_state=42)

In [5]:
# define placeholders
features = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(features, [227, 227])
labels = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(labels, 43)

In [6]:
# pass placeholders to alexet
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

In [7]:
# Add the final layer for traffic sign classification.
nb_classes = 43
shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
fc8W = tf.Variable(tf.truncated_normal(shape = shape, mean = 0, stddev = 0.01))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

In [8]:
# Define loss, training, accuracy operations.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss_op, var_list=[fc8W, fc8b])
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
# Get the reduced data keeping class distribution of original data set
def get_sample(X_data, y_data, sample_size):
    idx = np.random.choice(range(len(y_data)), sample_size)
    return X_data[idx], y_data[idx]
n_mini_train = 500
n_mini_valid = 100
X_mini_train, y_mini_train = get_sample(X_train, y_train, n_mini_train)
X_mini_train, y_mini_train = X_train, y_train

In [10]:
# Function to measure accuracy
def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], BATCH_SIZE):
        end = offset + BATCH_SIZE
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

In [11]:
# Train and evaluate the feature extraction model.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(EPOCHS):
        # training
        X_mini_train, y_mini_train = shuffle(X_mini_train, y_mini_train)
        t0 = time.time()
        for offset in range(0, X_mini_train.shape[0], BATCH_SIZE):
            end = offset + BATCH_SIZE
            sess.run(training_op, feed_dict={features: X_mini_train[offset:end], labels: y_mini_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_valid, y_valid, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")

Epoch 1
Time: 111.784 seconds
Validation Loss = 0.509178673923
Validation Accuracy = 0.858837031415

Epoch 2
Time: 107.720 seconds
Validation Loss = 0.322441880814
Validation Accuracy = 0.909589390477

Epoch 3
Time: 107.760 seconds
Validation Loss = 0.240138415204
Validation Accuracy = 0.941341494532

Epoch 4
Time: 107.745 seconds
Validation Loss = 0.191359273373
Validation Accuracy = 0.955878602413

Epoch 5
Time: 107.738 seconds
Validation Loss = 0.178006540597
Validation Accuracy = 0.955113491471

Epoch 6
Time: 107.762 seconds
Validation Loss = 0.162049114993
Validation Accuracy = 0.958684009318

Epoch 7
Time: 107.725 seconds
Validation Loss = 0.133780863908
Validation Accuracy = 0.967100229548

Epoch 8
Time: 107.748 seconds
Validation Loss = 0.124181408484
Validation Accuracy = 0.96786534049

Epoch 9
Time: 107.752 seconds
Validation Loss = 0.120120625925
Validation Accuracy = 0.967355266529

Epoch 10
Time: 107.748 seconds
Validation Loss = 0.117501498762
Validation Accuracy = 0.9677